<a href="https://colab.research.google.com/github/lazy-wav/Data-Analysis/blob/main/DistilBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers
!pip install --upgrade transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 45.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.3
    Uninstalling transformers-4.52.3:
      Successfully uninstalled transformers-4.52.3


In [3]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from tqdm import tqdm

from transformers import DistilBertTokenizer

from transformers import DistilBertForSequenceClassification

In [4]:
from google.colab import files
uploaded = files.upload()

Saving Fake.csv to Fake.csv


In [5]:
from google.colab import files
uploaded = files.upload()

Saving True.csv to True.csv


In [6]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments
from datasets import Dataset
import numpy as np

# 1. Load data and label


In [7]:
true_data = pd.read_csv('True.csv')
fake_data = pd.read_csv('Fake.csv')
true_data['label'] = 0
fake_data['label'] = 1
data = pd.concat([true_data, fake_data]).sample(frac=1).reset_index(drop=True)

In [8]:
texts = data['title'].tolist()
labels = data['label'].tolist()


# 2. Train/Val/Test split


In [9]:
train_texts, temp_texts, train_labels, temp_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)
val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts, temp_labels, test_size=0.5, random_state=42)

# 3. Tokenizer


In [10]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

def tokenize_fn(texts):
    return tokenizer(texts, truncation=True, padding=True, max_length=64)

train_encodings = tokenize_fn(train_texts)
val_encodings = tokenize_fn(val_texts)
test_encodings = tokenize_fn(test_texts)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

# 4. Convert to Hugging Face Dataset format

In [11]:

train_dataset = Dataset.from_dict({**train_encodings, 'labels': train_labels})
val_dataset = Dataset.from_dict({**val_encodings, 'labels': val_labels})
test_dataset = Dataset.from_dict({**test_encodings, 'labels': test_labels})

# 5. Load Model

In [12]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# 6. Define metric function


In [13]:

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = (predictions == labels).mean()
    return {'accuracy': accuracy}

# 7. Training arguments


In [14]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir='./logs',
)


In [16]:
trainer.evaluate(test_dataset)


{'eval_loss': 0.055315230041742325,
 'eval_accuracy': 0.9835189309576837,
 'eval_runtime': 542.8198,
 'eval_samples_per_second': 8.272,
 'eval_steps_per_second': 0.26,
 'epoch': 1.0}

In [17]:
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=64)
    inputs = {key: val.to(model.device) for key, val in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        predicted_class = torch.argmax(outputs.logits, dim=1).item()
    return "FAKE" if predicted_class == 1 else "REAL"

# Example:
print(predict("Breaking: Scientists discover new planet in our solar system"))


FAKE


In [18]:
predictions_output = trainer.predict(test_dataset)
logits = predictions_output.predictions
true_labels = predictions_output.label_ids
pred_labels = np.argmax(logits, axis=1)


In [19]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

print("Accuracy:", accuracy_score(true_labels, pred_labels))
print("\nClassification Report:")
print(classification_report(true_labels, pred_labels, target_names=["Real", "Fake"]))
print("\nConfusion Matrix:")
print(confusion_matrix(true_labels, pred_labels))


Accuracy: 0.9835189309576837

Classification Report:
              precision    recall  f1-score   support

        Real       0.98      0.99      0.98      2064
        Fake       0.99      0.98      0.98      2426

    accuracy                           0.98      4490
   macro avg       0.98      0.98      0.98      4490
weighted avg       0.98      0.98      0.98      4490


Confusion Matrix:
[[2040   24]
 [  50 2376]]


In [20]:
import shutil
import os
import zipfile

# Save the model locally
save_path = './saved_model'
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

# Zip the folder
zip_path = shutil.make_archive('saved_model', 'zip', save_path)

# Download the zipped model
from google.colab import files
files.download(zip_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [22]:
save_path = '/content/drive/MyDrive/saved_model'
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print(f"Model saved to: {save_path}")


Model saved to: /content/drive/MyDrive/saved_model
